In [ ]:
%pip install pyautogen==0.2.3

In [15]:
import autogen
import os
config_list = [
    {
        'model': 'gpt-4',
        'api_key': os.environ.get('OPENAI_KEY'),
    },
]
llm_config = {"temperature": 0.1, "config_list": config_list, "cache_seed": 4, "timeout": 600}

class Auctioneer(autogen.AssistantAgent):
    pass
class Bidder(autogen.AssistantAgent):
    pass

In [18]:
import autogen
from typing import Optional, List, Dict, Any

class AuctionGC(autogen.GroupChat):
    def __init__(self, agents, messages, max_round=10):
        super().__init__(agents, messages, max_round)
        self.previous_speaker = None  
        self.speaker_idx = 0
        self.speaker_selection_method = 'round_robin'

    def select_speaker(self, last_speaker, selector):
        self.previous_speaker = last_speaker
        self.speaker_idx = self.speaker_idx + 1
        if self.speaker_idx >= len(self.agents):
            self.speaker_idx = 0
        return self.agents[self.speaker_idx]

class Auctioneer(autogen.AssistantAgent):
    highest_bid: int
    winner: str
    bidders: List["Bidder"]
    
    def __init__(self, auction_type, llm_config=llm_config, **kwargs):
        super().__init__(
            name="Auctioneer",
            system_message=self._get_auction_sys_msg(auction_type),
            llm_config=llm_config
        )
        self.auction_type = auction_type
        self.register_reply("Bidder", self._highest_bid_reply)
    
    def _get_auction_sys_msg(self, auction_type):
        if auction_type == "Dutch":
            return """You are an auctioneer selling an item using a Dutch Auction.
                      Start with a high price and gradually decrease it until a bidder accepts.
                      In case a bidder accepts, send out the 'END OF AUCTION' message."""
        elif auction_type == "English":
            return """You are an auctioneer selling an item using an English Auction.
                      Bidders openly compete by placing progressively higher bids.
                      In case nobody or only one bidder raised in the latest round, send out the 'END OF AUCTION' message."""
        elif auction_type == "Sealed-Bid":
            return """You are an auctioneer selling an item using a Sealed-Bid Auction.
                      Bidders submit their bids privately in 1 round, highest bid wins.
                      Evalute the bids, send back the winner, highest bid and the 'END OF AUCTION' message."""
        elif auction_type == "Vickrey":
            return """You are an auctioneer selling an item using a Vickrey Auction.
                      Bidders submit their bids privately in 1 round, highest bid wins, but only pays the second-highest bid.
                      Evalute the bids and send back the winner, the highest bid, the price to be paid and the 'END OF AUCTION' message."""
        else:
            raise ValueError(f"Invalid auction type: {auction_type}")

    def _highest_bid_reply(self, messages: Optional[List[Dict]] = None, sender: Optional[autogen.Agent] = None, config: Optional[Any] = None):
        message = self.generate_reply(messages, sender, exclude=[self._highest_bid_reply])
        if message is None:
            return True, None
        else:
            return True, message


class Bidder(autogen.AssistantAgent):
    auctioneer: Auctioneer
    bidder_id: int
    
    def __init__(self, _id, _auctioneer, auction_type, llm_config=llm_config, **kwargs):
        super().__init__(
            name=f"Bidder{_id}",
            system_message=self._get_bidder_sys_msg(
                bidder_id=_id,
                bidder_increment=5,
                bidder_limit=50,
                auction_type=auction_type
            ),
            is_termination_msg=self._end_of_auction,
            llm_config=llm_config
        )
        self.bidder_id = _id
        self.auctioneer = _auctioneer
        self.register_reply(Auctioneer, self._decide_bid)

    def _end_of_auction(self, msg):
        return 'END OF AUCTION' in msg['content']
    
    def _decide_bid(self, messages: Optional[List[Dict]] = None, sender: Optional[autogen.Agent] = None, config: Optional[Any] = None):
        message = self.generate_reply(messages, sender, exclude=[self._decide_bid])
        if message is None:
            print("No message for bidder " + str(self.bidder_id ))
            return True, None
        else:
            return True, message

    def _get_bidder_sys_msg(self, bidder_id, bidder_increment, bidder_limit, auction_type):
        if auction_type == "Dutch":
            return f"""You are a bidder at a Dutch Auction looking to buy an item. Your id is {bidder_id}.
                       To accept the current offer, it should be less than {bidder_limit}.
                       When you accept the current offer, you send your id followed by the 'ACCEPT' message.
                       When you don't want to accept the offer, you send your id followed by the 'PASS' message.
                       If another bidder already accepted the offer, you cannot accept the offer and say 'Great Deal, Goodbye!'."""
        elif auction_type == "English":
            return f"""You are a bidder at an English Auction looking to buy an item. Your id is {bidder_id}.
                       You always increase by {bidder_increment} and should not exceed {bidder_limit} with your bids.
                       When you raise the bid, you should reply with your id followed by the new amount proposed as plain text without any other symbol.
                       When you don't want to participate anymore, you send your id followed by the 'QUIT' message in plain text without any other symbol.
                    """
        elif auction_type == "Sealed-Bid":
            return f"""You are a bidder at a Sealed-Bid Auction looking to buy an item. Your id is {bidder_id}.
                       You need to submit your bid once, by replying with your id followed by the proposed price.
                       You must not consider other bidders' proposals, only consider the value proposed by the auctioneer to ensure private voting.
                    """
        elif auction_type == "Vickrey":
            return f"""You are a bidder at a Vickrey Auction looking to buy an item. Your id is {bidder_id}.
                       You need to submit your bid once, by replying with your id followed by the proposed price.
                       You must not consider other bidders' proposals, only consider the value proposed by the auctioneer to ensure private voting.
                    """
        else:
            raise ValueError(f"Invalid auction type: {auction_type}")

In [20]:
auction_type = "Vickrey"  
auctioneer = Auctioneer(auction_type, llm_config={"temperature": 0.1, "cache_seed": 4, "config_list": config_list, "timeout": 600})
bidder = Bidder(_id=1, _auctioneer=auctioneer, auction_type=auction_type)
bidder1 = Bidder(_id=2, _auctioneer=auctioneer, auction_type=auction_type)
#bidder2 = Bidder(_id=3, _auctioneer=auctioneer, auction_type=auction_type)

# groupchat = autogen.GroupChat(agents=[auctioneer, bidder, bidder1], messages=[])
groupchat = AuctionGC(agents=[auctioneer, bidder, bidder1], messages=[], max_round=100)
manager = autogen.GroupChatManager(name="Auction Chat", groupchat=groupchat, llm_config=llm_config)
#auctioneer.initiate_chat(manager, message="Initial price for the item is 100 euros, do you want to accept it?")
auctioneer.initiate_chat(manager, message="Value of the item is 100 euros.")

Auctioneer (to Auction Chat):

Value of the item is 100 euros.

--------------------------------------------------------------------------------
Bidder1 (to Auction Chat):

1, 120 euros

--------------------------------------------------------------------------------
Bidder2 (to Auction Chat):

2, 110 euros

--------------------------------------------------------------------------------
Auctioneer (to Auction Chat):

The winner is Bidder 1 with the highest bid of 120 euros. However, according to the rules of the Vickrey Auction, Bidder 1 will only pay the second highest bid, which is 110 euros. 

END OF AUCTION.

--------------------------------------------------------------------------------
